In [11]:
# fm training for anime

import xlearn as xl
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold


In [12]:
# 读取 csv
anime_train_df = pd.read_csv("data/anime_genres_splitted.csv" , header=None)
# anime_train_df = anime_train_df.drop(labels=['type'],axis=1)

col_num = len(anime_train_df.columns.tolist())
print(col_num)

45


In [13]:
# 采样 sampling
x = anime_train_df
kf = KFold(n_splits=5,shuffle=True)
for train_index, test_index in kf.split(x):
  print(train_index,test_index)
print(len(train_index))
print(len(test_index))

[    0     1     3 ... 11951 11952 11953] [    2     5     9 ... 11945 11949 11950]
[    0     1     2 ... 11950 11951 11953] [   10    12    14 ... 11940 11943 11952]
[    0     1     2 ... 11949 11950 11952] [    3     4     8 ... 11947 11951 11953]
[    1     2     3 ... 11951 11952 11953] [    0     6    26 ... 11942 11944 11946]
[    0     2     3 ... 11951 11952 11953] [    1     7    13 ... 11925 11930 11948]
9564
2390


In [14]:
train_sample = anime_train_df.drop(test_index)
test_sample = anime_train_df.drop(train_index)

X_train = train_sample[train_sample.columns[1:]]
y_train = train_sample[0]
# print(train_sample)

X_test = test_sample[test_sample.columns[1:]]
y_test = test_sample[0]
# test_sample

xdm_train = xl.DMatrix(X_train, y_train)
xdm_test = xl.DMatrix(X_test, y_test)


In [15]:
# 开始 train
# Training task
fm_model = xl.create_fm()  # Use factorization machine
# we use the same API for train from file
# that is, you can also pass xl.DMatrix for this API now
fm_model.setTrain(xdm_train)    # Training data
fm_model.setValidate(xdm_test)  # Validation data

# param:
#  0. regression task
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: acc
param = {'task':'binary', 'lr':0.2, 
         'lambda':0.002, 'metric':'acc'}

# Start to train
# The trained model will be stored in model.out
fm_model.fit(param, './model_dm.out')

# Prediction task
# we use the same API for test from file
# that is, you can also pass xl.DMatrix for this API now
fm_model.setTest(xdm_test)  # Test data
fm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
# if no result out path setted, we return res as numpy.ndarray
res = fm_model.predict("./model_dm.out", "./output.txt")

